In [ ]:
!pip install -q transformers datasets seqeval evaluate accelerate

from pathlib import Path
import json
import os
from datasets import Dataset, DatasetDict, concatenate_datasets
import numpy as np

from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
)

import evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00


In [ ]:
def load_jsonl_to_examples(path, token_field="target_tokens", tag_field="target_tags"):
    examples = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            tokens = obj.get(token_field)
            tags = obj.get(tag_field)
            if tokens is None or tags is None:
                # try alternate keys
                tokens = obj.get("target_tokens") or obj.get("tokens")
                tags = obj.get("target_tags") or obj.get("ner_tags")
            if tokens and tags and len(tokens) == len(tags):
                examples.append({"tokens": tokens, "tags": tags})
    return examples

In [ ]:
def flat_to_bio(tags):
    bio = []
    prev = "O"
    for t in tags:
        if t == "O" or t.upper() == "O":
            bio.append("O")
            prev = "O"
        else:
            # treat same entity sequence as continuation
            if prev == t:
                bio.append("I-" + t)
            else:
                bio.append("B-" + t)
            prev = t
    return bio


In [ ]:
base_dir = "."   # or "." if files are in current working dir

lang_files = {
    "as": f"{base_dir}/projected_hi_as_ner.jsonl",
    "bn": f"{base_dir}/projected_hi_bn_ner.jsonl",
    "gu": f"{base_dir}/projected_hi_gu_ner.jsonl",
    "ml": f"{base_dir}/projected_hi_ml_ner.jsonl",
    "mr": f"{base_dir}/projected_hi_mr_ner.jsonl",
    "ta": f"{base_dir}/projected_hi_ta_ner.jsonl",
    "te": f"{base_dir}/projected_hi_te_ner.jsonl",
    # If you also have a Hindi file, add it here. If not, remove the 'hi' key.
    # "hi": f"{base_dir}/projected_hi_hi_ner.jsonl",
}

# quick sanity check: print which files exist
import os
for lang, p in lang_files.items():
    print(lang, "->", p, ":", "OK" if os.path.exists(p) else "MISSING")


as -> ./projected_hi_as_ner.jsonl : OK
bn -> ./projected_hi_bn_ner.jsonl : OK
gu -> ./projected_hi_gu_ner.jsonl : OK
ml -> ./projected_hi_ml_ner.jsonl : OK
mr -> ./projected_hi_mr_ner.jsonl : OK
ta -> ./projected_hi_ta_ner.jsonl : OK
te -> ./projected_hi_te_ner.jsonl : OK


In [ ]:
all_examples = []
for lang, p in lang_files.items():
    exs = load_jsonl_to_examples(p, token_field="target_tokens", tag_field="target_tags")
    # convert to BIO tags
    for e in exs:
        e["tags"] = flat_to_bio(e["tags"])
        e["lang"] = lang
    all_examples.extend(exs)

print("Loaded examples:", len(all_examples))

# train/validation/test 80/10/10
from sklearn.model_selection import train_test_split
train_and_val, test = train_test_split(all_examples, test_size=0.10, random_state=42)
train, val = train_test_split(train_and_val, test_size=0.1111, random_state=42)  # ~0.10 of total

ds_train = Dataset.from_list(train)
ds_val = Dataset.from_list(val)
ds_test = Dataset.from_list(test)

dataset = DatasetDict({"train": ds_train, "validation": ds_val, "test": ds_test})
print({k: len(v) for k, v in dataset.items()})

Loaded examples: 144
{'train': 114, 'validation': 15, 'test': 15}


In [ ]:
# collect unique labels across train set
unique_labels = set()
for tags in dataset["train"]["tags"]:
    unique_labels.update(tags)
unique_labels = sorted(list(unique_labels))  # deterministic
print("Unique labels (BIO):", unique_labels)

label2id = {lab: i for i, lab in enumerate(unique_labels)}
id2label = {i: lab for lab, i in label2id.items()}
num_labels = len(label2id)
print("Num labels:", num_labels)


Unique labels (BIO): ['B-LOC', 'B-ORG', 'B-PER', 'I-LOC', 'I-ORG', 'I-PER', 'O']
Num labels: 7


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "Davlan/xlm-roberta-base-wikiann-ner"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# safe load: allow mismatched sizes (will random-init head)
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    ignore_mismatched_sizes=True    # <--- key flag
)

# set label maps
model.config.id2label = id2label
model.config.label2id = label2id

# sanity: check classifier output dim
for n, p in model.named_parameters():
    if "classifier" in n or "score" in n:
        print(n, p.shape)


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at Davlan/xlm-roberta-base-wikiann-ner and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([7]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


classifier.weight torch.Size([7, 768])
classifier.bias torch.Size([7])


In [ ]:
def tokenize_and_align_labels(batch):
    tokenized = tokenizer(
        batch["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding=False
    )
    all_labels = []
    for i, labels in enumerate(batch["tags"]):
        word_ids = tokenized.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                label = labels[word_idx]
                # assign the same label to subwords; change to -100 to ignore
                label_ids.append(label2id[label])
            previous_word_idx = word_idx
        all_labels.append(label_ids)
    tokenized["labels"] = all_labels
    return tokenized

# apply mapping
tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=["tokens", "tags", "lang"]
)


Map:   0%|          | 0/114 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [ ]:
seqeval = evaluate.load("seqeval")

def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)

    true_predictions = []
    true_labels = []

    for p, l in zip(preds, labels):
        cur_preds = []
        cur_labels = []
        for pred_id, label_id in zip(p, l):
            if label_id == -100:
                continue
            cur_preds.append(id2label[int(pred_id)])
            cur_labels.append(id2label[int(label_id)])
        true_predictions.append(cur_preds)
        true_labels.append(cur_labels)

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results.get("overall_accuracy", 0.0),
    }


In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

training_args = TrainingArguments(
    output_dir="./xlm-roberta-multilang-ner",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=100,
    push_to_hub=False,
    fp16=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-174561587.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()
trainer.save_model("./xlm-roberta-multilang-ner")
tokenizer.save_pretrained("./xlm-roberta-multilang-ner")


HindiTimeBank Finetuning

In [ ]:
import xml.etree.ElementTree as ET
from pathlib import Path
import json

def parse_timebank_xml(xml_path):
    """
    Parse TimeBank XML and extract tokens with BIO tags.
    Returns: (tokens_list, bio_tags_list)
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Extract tokens (sorted by id)
    token_dict = {}
    for tok in root.findall("token"):
        tok_id = int(tok.attrib["id"])
        tok_text = tok.text.strip() if tok.text else ""
        token_dict[tok_id] = tok_text

    # Create initial labels (all O)
    num_tokens = len(token_dict)
    labels = ["O"] * num_tokens

    # Process EVENT tags
    for event in root.findall("EVENT"):
        token_ids = event.attrib["tokens"].split(",")
        for tid in token_ids:
            idx = int(tid) - 1  # Convert to 0-based
            if 0 <= idx < num_tokens:
                labels[idx] = "EVENT"

    # Process STATE tags
    for state in root.findall("STATE"):
        token_ids = state.attrib["tokens"].split(",")
        for tid in token_ids:
            idx = int(tid) - 1
            if 0 <= idx < num_tokens:
                labels[idx] = "STATE"

    # Process TIMEX tags
    for timex in root.findall("TIMEX"):
        token_ids = timex.attrib["tokens"].split(",")
        for tid in token_ids:
            idx = int(tid) - 1
            if 0 <= idx < num_tokens:
                labels[idx] = "TIMEX"

    # Convert to ordered lists
    tokens = [token_dict[i+1] for i in range(num_tokens)]

    # Convert flat labels to BIO format
    bio_labels = flat_to_bio(labels)

    return tokens, bio_labels

def flat_to_bio(labels):
    """Convert flat labels (EVENT, STATE, TIMEX, O) to BIO format."""
    bio = []
    prev_label = "O"

    for label in labels:
        if label == "O":
            bio.append("O")
            prev_label = "O"
        else:
            # Check if continuing same entity
            if label == prev_label:
                bio.append(f"I-{label}")
            else:
                bio.append(f"B-{label}")
            prev_label = label

    return bio

# Test on single file
xml_path = "HindiTimeBankData/xml/841.xml"
tokens, bio_labels = parse_timebank_xml(xml_path)

print("✓ Parsing test successful!")
print(f"Tokens (first 10): {tokens[:10]}")
print(f"BIO tags (first 10): {bio_labels[:10]}")
print(f"\nSample EVENT tokens:")
for i, (tok, tag) in enumerate(zip(tokens, bio_labels)):
    if "EVENT" in tag:
        print(f"  {tok}: {tag}")
        if i > 5:  # Show only first few
            break


✓ Parsing test successful!
Tokens (first 10): ['PMC', 'बैंक', 'के', '4300', 'करोड़', 'रुपये', 'के', 'घोटाले', 'की', 'जांच']
BIO tags (first 10): ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EVENT', 'O', 'B-STATE']

Sample EVENT tokens:
  घोटाले: B-EVENT


In [ ]:
from pathlib import Path
from tqdm import tqdm

def load_all_timebank_data(base_dir="HindiTimeBankData"):
    """
    Load all XML files from HindiTimeBankData folder.
    Returns list of examples: [{"tokens": [...], "tags": [...]}, ...]
    """
    xml_dir = Path(base_dir) / "xml"

    if not xml_dir.exists():
        print(f" Directory not found: {xml_dir}")
        return []

    xml_files = list(xml_dir.glob("*.xml"))
    print(f"Found {len(xml_files)} XML files")

    examples = []
    errors = []

    for xml_file in tqdm(xml_files, desc="Processing XML files"):
        try:
            tokens, bio_labels = parse_timebank_xml(xml_file)

            # Validate
            if len(tokens) == len(bio_labels) and len(tokens) > 0:
                examples.append({
                    "tokens": tokens,
                    "tags": bio_labels,
                    "file_id": xml_file.stem
                })
            else:
                errors.append(xml_file.name)
        except Exception as e:
            errors.append(f"{xml_file.name}: {str(e)}")

    print(f"\n✓ Successfully processed: {len(examples)} files")
    if errors:
        print(f" Errors in {len(errors)} files:")
        for err in errors[:5]:  # Show first 5
            print(f"    {err}")

    return examples

# Load all data
all_examples = load_all_timebank_data("HindiTimeBankData")

# Show statistics
print("\n" + "="*60)
print("📊 DATASET STATISTICS")
print("="*60)

total_tokens = sum(len(ex["tokens"]) for ex in all_examples)
print(f"Total examples: {len(all_examples)}")
print(f"Total tokens: {total_tokens}")

# Count label distribution
from collections import Counter
all_tags = []
for ex in all_examples:
    all_tags.extend(ex["tags"])

tag_counts = Counter(all_tags)
print(f"\nLabel distribution:")
for tag, count in sorted(tag_counts.items()):
    pct = count / len(all_tags) * 100
    print(f"  {tag:<15} {count:>6} ({pct:>5.2f}%)")

print("\n✅ Step 2 complete!")


In [ ]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# Create train/val/test splits (80/10/10)
train_val, test_examples = train_test_split(all_examples, test_size=0.10, random_state=42)
train_examples, val_examples = train_test_split(train_val, test_size=0.1111, random_state=42)  # ~10% of total

print(f"Train: {len(train_examples)} examples")
print(f"Val:   {len(val_examples)} examples")
print(f"Test:  {len(test_examples)} examples")

# Create HuggingFace datasets
ds_train = Dataset.from_list(train_examples)
ds_val = Dataset.from_list(val_examples)
ds_test = Dataset.from_list(test_examples)

dataset = DatasetDict({
    "train": ds_train,
    "validation": ds_val,
    "test": ds_test
})

# Create label mappings
unique_labels = sorted(list(set(all_tags)))
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}
num_labels = len(label2id)


print(f"\n✓ Unique labels ({num_labels}):")
for label in unique_labels:
    print(f"  {label}")

print("\n✅ Step 3 complete!")


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Use XLM-RoBERTa (good for Hindi)
model_name = "xlm-roberta-base"

print("Loading tokenizer and model...")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Load model with our label configuration
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True  # Allow head reinitialization
)

print(f"✓ Model loaded with {num_labels} labels")
print(f"✓ Tokenizer loaded")

# Verify model config
print(f"\nModel config:")
print(f"  Labels: {list(model.config.id2label.values())}")
print(f"  Classifier output dim: {model.classifier.out_features}")

print("\n✅ Step 4 complete!")


In [ ]:
def tokenize_and_align_labels(examples):
    """
    Tokenize tokens and align labels to subword tokens.
    """
    tokenized_inputs = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding=False,
        max_length=512
    )

    labels = []
    for i, label_list in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None

        for word_idx in word_ids:
            if word_idx is None:
                # Special tokens get -100
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                # First subword token gets the label
                label_ids.append(label2id[label_list[word_idx]])
            else:
                # Subsequent subword tokens get -100 (or same label)
                label_ids.append(-100)

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply tokenization
print("Tokenizing dataset...")
tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=["tokens", "tags", "file_id"]
)

print("\n✓ Tokenization complete!")
print(f"Train samples: {len(tokenized_dataset['train'])}")
print(f"Val samples: {len(tokenized_dataset['validation'])}")
print(f"Test samples: {len(tokenized_dataset['test'])}")

# Show sample
sample = tokenized_dataset["train"][0]
print(f"\nSample tokenized example:")
print(f"  Input IDs length: {len(sample['input_ids'])}")
print(f"  Labels length: {len(sample['labels'])}")
print(f"  First 10 labels: {sample['labels'][:10]}")

print("\n✅ Step 5 complete!")


In [ ]:
!pip install -q evaluate seqeval

# Then run the code again
from transformers import DataCollatorForTokenClassification, TrainingArguments, Trainer
import evaluate
import numpy as np

# Data collator for padding
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Load seqeval metric
seqeval = evaluate.load("seqeval")

def compute_metrics(pred):
    """Compute precision, recall, F1 for event extraction."""
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=2)

    # Convert to label strings (ignore -100)
    true_predictions = []
    true_labels = []

    for prediction, label in zip(predictions, labels):
        pred_labels = []
        true_label_list = []

        for pred_id, label_id in zip(prediction, label):
            if label_id != -100:
                pred_labels.append(id2label[pred_id])
                true_label_list.append(id2label[label_id])

        true_predictions.append(pred_labels)
        true_labels.append(true_label_list)

    # Compute metrics
    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results.get("overall_accuracy", 0.0),
    }

# Training arguments
training_args = TrainingArguments(
    output_dir="./hindi-timebank-event-model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
    fp16=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("✓ Trainer initialized")
print(f"\nTraining configuration:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Train samples: {len(tokenized_dataset['train'])}")

print("\n✅ Step 6 complete! Ready to train.")


In [ ]:
# Disable wandb logging
import os
os.environ["WANDB_DISABLED"] = "true"

# Recreate trainer without wandb
training_args = TrainingArguments(
    output_dir="./hindi-timebank-event-model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
    fp16=False,
    report_to="none",  # Disable all logging
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("✓ Trainer re-initialized without wandb")
print("\n Starting training...")

# Train
trainer.train()

print("\n Training complete!")

# Save model
trainer.save_model("./hindi-timebank-event-model-final")
tokenizer.save_pretrained("./hindi-timebank-event-model-final")

print(" Model saved!")



In [ ]:
print(" Evaluating on test set...\n")

# Evaluate
test_results = trainer.evaluate(tokenized_dataset["test"])

print("="*60)
print(" TEST SET RESULTS")
print("="*60)
print(f"\nPrecision: {test_results['eval_precision']:.3f}")
print(f"Recall:    {test_results['eval_recall']:.3f}")
print(f"F1 Score:  {test_results['eval_f1']:.3f}")
print(f"Accuracy:  {test_results['eval_accuracy']:.3f}")

print("\n Evaluation complete!")
